In [1]:
import numpy as np
from deepsudoku import REPO_PATH
import os
import glob
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from pathlib import Path
import tqdm
import tensorflow as tf
from deepsudoku.supervised_learning import SudukoWrapper, load_supervised_data

os.chdir(REPO_PATH)

2023-09-07 09:53:35.053387: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-09-07 09:53:35.675355: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [2]:
df = load_supervised_data.get_data("test", "realistic_easy_data")

In [3]:
names = os.listdir("supervised_experiments/logs")
comp = [n for n in names if "same" in (n or n[0]=="b") and "trans" not in n] + ["big_conv_head_1693994852.9515347"]
comp

name = "same_fc_mlp_1693909990.6347742"

In [4]:
def get_df(name):

    preds = np.load(os.path.join("stats/preds",f"{name}.npy"), )
    pred_df = df.copy()
    pred_df["preds"] = preds
    pred_df["win"] = pred_df.preds ==pred_df.solution

    def overlap(str1,str2):

        count = 0
        for i in range(81):
            if str1[i] == str2[i]:
                count += 1
        return count /81

    pred_df["accuracy"] =  [overlap(a,b) for a,b in tqdm.tqdm(zip(pred_df.preds ,pred_df.solution))]

    pred_df["run"] = "".join([i for i in name.replace(".","") if not i.isdigit()]).strip("_")

    return pred_df[["missing_digits","win","accuracy","run"]]

dfs = []
for name in comp:
    dfs.append(get_df(name))
df = pd.concat(dfs)
df.to_csv("stats/dl_architectures_test_results.csv")

1080000it [00:04, 227971.44it/s]
1080000it [00:04, 230032.44it/s]
1080000it [00:04, 229126.14it/s]
1080000it [00:04, 225461.20it/s]
1080000it [00:04, 244670.91it/s]
